In [2]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# Load dataset

In [1]:
def load_data(filename):
    '''
    Load in data and return values
    
    param:   filename (str) - filename/path to data
    return:  data - data values
    '''
    # double check it's samples x features
    data = pd.read_csv(filename)
    return data.values

In [ ]:
def norm_data(data):
    '''
    Normalize data to z-values (0 mean and 1 std dev)
    
    param:   data - data values (n_samples, n_features)
    return:  data - data values
    '''